# Sentiment Classification of Restaurant Reviews using OpenAI LLM

#### Function to classify the sentiment of a restaurant review using OpenAI API

#### This notebook leverages OpenAI's LLMs to classify the sentiment of restaurant reviews as positive, negative or neutral. It demonstrates how to use an API-based classification approach by sending review texts to a language model and interpreting its responses. 

#### The goal is to generate reliable sentiment labels for supervised learning or evaluation tasks. The notebook includes a wrapper function for model calls, test examples, and dataset preprocessing for bulk classification.

In [ ]:
# Importing function from utils.py to make OpenAI API calls
from utils import call_openai





In [ ]:
# Sending a restaurant review to the OpenAI API and returning its sentiment classification.

    # Parameters:
    #- comment (str): A string containing the restaurant review.

    #Returns:
    #- str: The sentiment classification ("positive", "neutral", or "negative").

def target_gen(comment):
    prompt = f"Classify the sentiment of the following restaurant review as 'positive', 'neutral', or 'negative':\n\n{comment}"
    return call_openai(prompt)

In [ ]:
#  Testing a single example with a hardcoded prompt to check OpenAI's response
prompt = "Classify the sentiment of this review as 'positive', 'neutral', or 'negative':\n\nThe food was amazing but service was slow."
response = call_openai(prompt)
print(response)

Neutral


In [ ]:
import pandas as pd

In [ ]:
# # Loading the whole training dataset with flexible CSV reading options 
# to handle inconsistent formatting

df = pd.read_csv("desafio_DS/dataset_train.csv", sep=None,  #automatically detecting sep
    engine="python",  #accepting different seps
    quoting=3,  # ignoring quotes
    on_bad_lines="warn"  # warning without breaking
)

In [ ]:
print(df.columns)

Index(['|input', 'Unnamed: 1'], dtype='object')


In [ ]:
# Renaming column with pipe symbol to "input"

df = df.rename(columns={"|input": "input"})


## Redefine target generator with exception handling

In [ ]:
def target_gen(comment):
    prompt = f"Classify the sentiment of the following restaurant review as 'positive', 'neutral', or 'negative':\n\n{comment}"
    try:
        response = call_openai(prompt)
        return response
    except Exception as e:
        print(f"Comment with Error: {comment}\n{e}")
        return "I am not sure how to classify, need human annotator."

#### Apply classification function to dataset

In [ ]:
df["target"] = df["input"].apply(target_gen)



In [ ]:
print(df)

                                                 input Unnamed: 1    target
0    0|judging from previous posts this used to be ...        NaN  Negative
1    1|we , there were four of us , arrived at noon...        NaN  Negative
2    2|they never brought us complimentary noodles ...        NaN  Negative
3    3|the food was lousy - too sweet or too salty ...        NaN  Negative
4    4|after all that , they complained to me about...        NaN  Negative
..                                                 ...        ...       ...
646       646|mizu is the japenese find in grammercy .        NaN  positive
647  647|while their kitchen food is delicious , th...        NaN  positive
648  648|mizu is home to creative and unique rolls ...        NaN  positive
649  649|not only is the cuisine the best around , ...        NaN  positive
650  650|warning : you may find it difficult to din...        NaN  positive

[651 rows x 3 columns]


In [ ]:
# Separating input column from '|'
df[['id', 'text']] = df['input'].str.split('|', n=1, expand=True)
df = df.drop(columns=["input", "Unnamed: 1"])


In [ ]:
print(df)

       target   id                                               text
0    Negative    0  judging from previous posts this used to be a ...
1    Negative    1  we , there were four of us , arrived at noon -...
2    Negative    2  they never brought us complimentary noodles , ...
3    Negative    3  the food was lousy - too sweet or too salty an...
4    Negative    4  after all that , they complained to me about t...
..        ...  ...                                                ...
646  positive  646           mizu is the japenese find in grammercy .
647  positive  647  while their kitchen food is delicious , their ...
648  positive  648  mizu is home to creative and unique rolls not ...
649  positive  649  not only is the cuisine the best around , the ...
650  positive  650  warning : you may find it difficult to dine at...

[651 rows x 3 columns]


In [ ]:
df.to_csv("dataset_llm_judge.csv", index=False)
